In [2]:
import pandas as pd
import gzip
import json
import re
import datetime
import numpy as np

In [3]:
def get_data_from_date(specific_date):
    return all_data[(all_data["Date"] == end_date)]

In [4]:
class Coinmarketcap(object):
    @staticmethod
    def load_data(source_datapath):
        all_data = pd.read_csv(source_datapath, sep='\t', header=0, encoding="utf-8", dtype={'Market Cap': np.float64, 'Total Market Cap': np.float64})
        all_data["Market Cap Share"] = all_data["Market Cap"]/all_data["Total Market Cap"]
        return all_data

    def __init__(self, source_datapath):
        self.data = self.load_data(source_datapath)
    
    def get_coin_price(self, date, coin_symbol):
        try:
            return float(self.data[(self.data["Date"] == date) & (self.data["Symbol"] == coin_symbol)]["Price"].values[0])
        except:
            print("ERROR: price not found for coin %s (%s)" % (symbol, date))
            return 0

    def coins_to_usd(self, date, coins):
        amount_usd = 0
        for coin_symbol, amount in coins.items():
            coin_usd = self.get_coin_price(date, symbol)
            amount_usd += amount * coin_usd
        return amount_usd
    
    def buy_coin(self, date, symbol, amount_usd):
        return amount_usd/self.get_coin_price(date, symbol)
    
    def buy_coins(self, date, symbols, amount_usd, strategy="even"):
        symbols_length = len(symbols)
        coins = {}
        if strategy == "market_cap":
            coins_market_cap = {}
            total_market_cap = 0
            for coin_symbol in symbols:
                coins_market_cap[coin_symbol] = self.get_coin_market_cap(date, coin_symbol)
                total_market_cap += coins_market_cap[coin_symbol]
        for coin_symbol in symbols:
            coin_amount_usd = 0
            if strategy == "even":
                coin_amount_usd = (amount_usd/symbols_length)
            elif strategy == "market_cap":
                coin_amount_usd = (amount_usd*(coins_market_cap[coin_symbol]/total_market_cap))
            coins[coin_symbol] = self.buy_coin(date, coin_symbol, coin_amount_usd)
        return coins
    
    def sell_coins(self, date, coins):
        amount_usd = 0
        for coin_symbol, coin_amount in coins.items():
            coin_price = self.get_coin_price(date, coin_symbol)
            amount_usd += coin_price*coin_amount
        return amount_usd
    
    def get_coin_market_cap(self, date, coin_symbol):
        return float(self.data[(self.data["Date"] == date) & (self.data["Symbol"] == coin_symbol)]["Market Cap"].values[0])
            
    def get_positions_range(self, date, first_position, last_position):
        coins_data = self.data[(self.data["Date"] == end_date) & (self.data["Pos"] <= last_position) & (self.data["Pos"] >= first_position)]
        for index, _ in coins_data["Symbol"].iteritems():
            yield coins_data["Symbol"][index]
    
    def dates(self):
        return sorted(self.data.Date.unique())
    
    def print_returns(self, initial_amount, initial_date, final_amount, final_date):
        print("%s: %s USD" % (initial_date, round(initial_amount, 2)))
        print("%s: %s USD" % (end_date, round(final_amount, 2)))
        print("----------")
        print("%sX" % round(final_amount/initial_amount, 1))
        

In [5]:
class InvestStrategies(object):
    def __init__(self, market):
        self.market = market

# Initialize

In [6]:
market = Coinmarketcap("data/coinmarketcap_formatted.tsv")
all_dates = market.dates()
weeks_ago = 54
initial_amount = 1000
start_date = all_dates[-weeks_ago]
end_date = all_dates[-1]

# Strategies

In [7]:
print("With an initial of %s USD from %s to %s" % (initial_amount, start_date, end_date))

With an initial of 1000 USD from 2016-04-10 to 2017-04-23


## Buy and forget

### Buy first 5 coins evenly

In [8]:
symbols_to_buy = market.get_positions_range(start_date, 1, 5)
coins = market.buy_coins(start_date, list(symbols_to_buy), initial_amount, strategy="even")
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

2016-04-10: 1000 USD
2017-04-23: 5703.29 USD
----------
5.7X


### Buy 50% Bitcoin 50% Top(2-5)

In [9]:
symbols_to_buy = list(market.get_positions_range(start_date, 1, 1))
coins = market.buy_coins(start_date, symbols_to_buy, initial_amount/2, strategy="even")
symbols_to_buy = list(market.get_positions_range(start_date, 2, 5))
top_coins = market.buy_coins(start_date, symbols_to_buy, initial_amount/2, strategy="even")
coins = {**coins, **top_coins}
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

2016-04-10: 1000 USD
2017-04-23: 4657.87 USD
----------
4.7X


### Buy Top 5 by market cap

In [10]:
symbols_to_buy = market.get_positions_range(start_date, 1, 5)
coins = market.buy_coins(start_date, list(symbols_to_buy), initial_amount, strategy="market_cap")
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

2016-04-10: 1000 USD
2017-04-23: 3280.62 USD
----------
3.3X


## Balancing every X week

### Buy first 5 coins evenly

In [14]:
coins = {}
for x_week in [1,4,8,12]:
    amount_to_buy = initial_amount
    for date in all_dates[-weeks_ago::x_week]:
        if date != start_date:
            amount_to_buy = market.sell_coins(date, coins)
        symbols_to_buy = market.get_positions_range(date, 1, 5)
        coins = market.buy_coins(date, list(symbols_to_buy), amount_to_buy, strategy="even")
    print("Balancing every %s weeks\n" % x_week)
    market.print_returns(initial_amount, start_date, amount_to_buy, end_date)

Balancing every 1 weeks

2016-04-10: 1000 USD
2017-04-23: 7308.92 USD
----------
7.3X
Balancing every 4 weeks

2016-04-10: 1000 USD
2017-04-23: 9021.08 USD
----------
9.0X
Balancing every 8 weeks

2016-04-10: 1000 USD
2017-04-23: 4516.01 USD
----------
4.5X
Balancing every 12 weeks

2016-04-10: 1000 USD
2017-04-23: 5463.12 USD
----------
5.5X


### Buy 50% Bitcoin 50% Top(2-5)

In [15]:
coins = {}
for x_week in [1,4,8,12]:
    for date in all_dates[-weeks_ago::x_week]:
        amount_to_buy = initial_amount
        if date != start_date:
            amount_to_buy = market.sell_coins(date, coins)
        symbols_to_buy = list(market.get_positions_range(start_date, 1, 1))
        coins = market.buy_coins(date, symbols_to_buy, amount_to_buy/2, strategy="even")
        symbols_to_buy = list(market.get_positions_range(start_date, 2, 5))
        top_coins = market.buy_coins(date, symbols_to_buy, amount_to_buy/2, strategy="even")
        coins = {**coins, **top_coins}
    print("Balancing every %s weeks\n" % x_week)
    market.print_returns(initial_amount, start_date, amount_to_buy, end_date)

Balancing every 1 weeks

2016-04-10: 1000 USD
2017-04-23: 5451.19 USD
----------
5.5X
Balancing every 4 weeks

2016-04-10: 1000 USD
2017-04-23: 6660.81 USD
----------
6.7X
Balancing every 8 weeks

2016-04-10: 1000 USD
2017-04-23: 3955.26 USD
----------
4.0X
Balancing every 12 weeks

2016-04-10: 1000 USD
2017-04-23: 4815.18 USD
----------
4.8X


### Buy Top 5 by market cap

In [16]:
coins = {}
for x_week in [1,4,8,12]:
    for date in all_dates[-weeks_ago::x_week]:
        amount_to_buy = initial_amount
        if date != start_date:
            amount_to_buy = market.sell_coins(date, coins)
        symbols_to_buy = market.get_positions_range(start_date, 1, 5)
        coins = market.buy_coins(date, list(symbols_to_buy), amount_to_buy, strategy="market_cap")
        amount_to_buy = market.sell_coins(date, coins)
    print("Balancing every %s weeks\n" % x_week)
    market.print_returns(initial_amount, start_date, amount_to_buy, end_date)

Balancing every 1 weeks

2016-04-10: 1000 USD
2017-04-23: 3308.3 USD
----------
3.3X
Balancing every 4 weeks

2016-04-10: 1000 USD
2017-04-23: 3214.02 USD
----------
3.2X
Balancing every 8 weeks

2016-04-10: 1000 USD
2017-04-23: 2616.16 USD
----------
2.6X
Balancing every 12 weeks

2016-04-10: 1000 USD
2017-04-23: 2615.46 USD
----------
2.6X
